In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df1 = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\DBs\ZPP Output_Processed\zppoutput_1s2022.csv", delimiter=';')
df2 = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\DBs\ZPP Output_Processed\zppoutput_2s2022.csv", delimiter=';')
df3 = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\DBs\ZPP Output_Processed\zppoutput_2s2021.csv", delimiter=';')
df4 = pd.read_csv(r"C:\Users\admin\Desktop\Projetos\DBs\ZPP Output_Processed\zppoutput_1s2023.csv", delimiter=';')

In [3]:
frames = [df1, df2, df3, df4]
df = pd.concat(frames)

In [4]:
df.to_csv('zppoutput.csv', sep=';', encoding='utf-8', index = False)